#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Homework 2:  PageRank + Learning to Rank

### 100 points [10% of your final grade]

### Due: March 5, 2020 by 11:59pm

*Goals of this homework:* In this homework you will explore real-world challenges of building a graph (in this case, from tweets), implement and test the classic PageRank algortihm over this graph. In addition, you will apply learning to rank to a real-world dataset and report the performance in terms of NDCG.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, my homework submission would be something like `555001234_hw2.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: PageRank (60 points)
In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. 


## Part 1.1: A re-Tweet Graph (20 points)

So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is retweeted by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

You should build a graph by parsing the tweets in the file we provide called *PageRank.json*.

**Notes:**

* You may see some weird characters in the content of tweets, just ignore them. 
* The edges are binary and directed. If Bob retweets Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [1]:
from collections import defaultdict

import copy
import operator
import json

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
dict_pagerank = {}

num_nodes = 0
num_edges = 0
d = 0.9

edge_in = defaultdict(set)
edge_out = defaultdict(set)

In [4]:
# Here define your function for building the graph by parsing 
# the input file of tweets
# Insert as many cells as you want

def graph_generate():
    global num_edges
    with open ('pagerank.json/hits.json',encoding="utf8") as d_file:
        for line in d_file:
            tweet = json.loads(line)
            user_id = tweet["user"]["id"]
            op_user_id = tweet["retweeted_status"]["user"]["id"]
            if user_id == op_user_id:
                continue
            edge_out[user_id].add(op_user_id)
            if op_user_id not in edge_out:
                edge_out[op_user_id] = set()
            edge_in[op_user_id].add(user_id)
            if user_id not in edge_in:
                edge_in[user_id] = set()

def print_graph_meta():
    global num_nodes
    global num_edges
    num_nodes = len(edge_out)
    print( "Total nodes: " + str(num_nodes))
    num_edges = 0
    for node in edge_out.keys():
        num_edges += len(edge_out[node])
    print ("Total Edges : " + str(num_edges))         

In [5]:
# Call your function to print out the size of the graph, 
# i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you 
#write the graph into a file, and load it later.
graph_generate()
print_graph_meta()

Total nodes: 1003
Total Edges : 6177


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## Part 1.2: PageRank Implementation (30 points)

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never retweeted and does not retweet anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

I check the page rank score of 50 nodes. If the page rank score dosent change in an iteration, we conclude that convergence has been achieved.

In [6]:
def get_topk(high):
    temp = 0
    for w in sorted(dict_pagerank, key=dict_pagerank.get, reverse=True):
        if temp == high:
            break
        print( w, dict_pagerank[w])
        temp += 1


In [7]:
def check_convergence(present, previous):
    count = 50
    if previous == None:
        return False
    for i in range(count):
        if present[i][0] != previous[i][0]:
            return False

    return True

In [8]:
# Here add your code to implement a function called PageRanker
# Insert as many cells as you want

def PageRanker():
    present = []
    previous = None
    for node in edge_out.keys():
        dict_pagerank[node] = 1.0 / num_nodes
    iter = 0
    while True:
        iter += 1
        temp_copy = copy.deepcopy(dict_pagerank)
        for node in edge_out.keys():
            dict_pagerank[node] = 0.0
            for incoming_node in edge_in[node]:
                total_len = len(edge_out[incoming_node])
                preempt_len = temp_copy[incoming_node]
                dict_pagerank[node] += float(preempt_len) / total_len
            dict_pagerank[node] *= d
            dict_pagerank[node] += (1 - d) / num_nodes
        present = sorted(dict_pagerank.items(), key = operator.itemgetter(1), reverse=True)
        flag = check_convergence(present, previous)
        if flag == True:
            break
        else:
            previous = present
    count_pr = sum(dict_pagerank.values())
    for key, value in dict_pagerank.items():
        dict_pagerank[key] = float(value)/count_pr
        

In [9]:
# Now let's call your function on the graph you've built. Output the results.
PageRanker()
get_topk(10)

1183906148 0.028884376712513107
3019659587 0.021570090805795614
3077695572 0.020951644836023964
3068694151 0.01850905491727246
2598548166 0.017946993839813852
3154266823 0.0174823129374862
3042570996 0.01737600593411384
571198546 0.017205680082834674
3039321886 0.015524345039655479
3082766914 0.01462300657234436


## Part 1.3: Improving PageRank (10 points)
In the many years since PageRank was introduced, there have been many improvements and extensions. For this part, you should experiment with one such improvement and then compare the results you get with the original results in Part 1.2. 

In [10]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar  7 16:38:08 2020

@author: rizuj
"""

from collections import defaultdict


import copy
import operator
import json

dict_pagerank_plus = {}

num_nodes_plus = 0
num_edges_plus = 0
d = 0.9

edge_in_plus = defaultdict(set)
edge_out_plus = defaultdict(set)

status_count = defaultdict(int)

# Here define your function for building the graph by parsing 
# the input file of tweets
# Insert as many cells as you want

def graph_generate_plus():
    global num_edges_plus
    with open ('pagerank.json/hits.json',encoding="utf8") as d_file:
        for line in d_file:
            tweet = json.loads(line)
            user_id = tweet["user"]["id"]
            op_user_id = tweet["retweeted_status"]["user"]["id"]
            if user_id == op_user_id:
                continue
            edge_out_plus[user_id].add(op_user_id)
            status_count[user_id] = tweet["user"]["statuses_count"]
            user_id = tweet["user"]["id"]

            if op_user_id not in edge_out_plus:
                edge_out_plus[op_user_id] = set()
            edge_in_plus[op_user_id].add(user_id)
            if user_id not in edge_in_plus:
                edge_in_plus[user_id] = set()

def print_graph_meta_plus():
    global num_nodes_plus
    global num_edges_plus
    num_nodes_plus = len(edge_out_plus)
    print( "Total nodes: " + str(num_nodes_plus))
    num_edges_plus = 0
    for node in edge_out_plus.keys():
        num_edges_plus += len(edge_out_plus[node])
    print ("Total Edges : " + str(num_edges_plus))             

def get_topk_plus(high):
    temp = 0
    for w in sorted(dict_pagerank_plus, key=dict_pagerank_plus.get, reverse=True):
        if temp == high:
            break
        print( w, dict_pagerank_plus[w])
        temp += 1


def check_convergence_plus(present, previous):
    count = 10
    if previous == None:
        return False
    for i in range(count):
        if present[i][0] != previous[i][0]:
            return False

    return True

def PageRanker_plus():
    present = []
    previous = None
    for node in edge_out_plus.keys():
        dict_pagerank_plus[node] = 1.0 / num_nodes_plus
    iter = 0
    max_status_count = max(status_count.values())
    while True:
        iter += 1
        temp_copy = copy.deepcopy(dict_pagerank_plus)
        for node in edge_out_plus.keys():
            dict_pagerank_plus[node] = 0.0
            for incoming_node in edge_in_plus[node]:
                total_len = len(edge_out_plus[incoming_node])
                preempt_len = temp_copy[incoming_node]
                dict_pagerank_plus[node] += float(preempt_len) / total_len
            dict_pagerank_plus[node] *= d
            dict_pagerank_plus[node] += (1 - d) / num_nodes_plus
            dict_pagerank_plus[node] += (0.5 + (status_count[node]/2*max_status_count))
        present = sorted(dict_pagerank_plus.items(), key = operator.itemgetter(1), reverse=True)
        flag = check_convergence_plus(present, previous)
        if flag == True:
            break
        else:
            previous = present
    count_pr = sum(dict_pagerank_plus.values())
    for key, value in dict_pagerank_plus.items():
        dict_pagerank_plus[key] = float(value)/count_pr
                

graph_generate_plus()
print_graph_meta_plus()
PageRanker_plus()
get_topk_plus(10)

Total nodes: 1003
Total Edges : 6177
3077695572 0.025246428474644228
3019659587 0.024718522012607137
571198546 0.024206629243985787
3068694151 0.021847448360243868
1183906148 0.021826427189460373
3039321886 0.01831122397752903
3082766914 0.01765680886983887
258262535 0.015297650674578228
1638625987 0.015112680338183256
3082049188 0.014436365553555925


#### Plus be sure to describe your extension (what is it? why did you choose it?) and your comparison to Part 1.2

The idea is that the user who gives a lot of statuses has potentially a lot of noise. So, in order to reduce noise and provide users with more quality feed, our pagerank algorithm should take into consideration the no. of tweets a person makes. To model this, we remember the status count of each node and after each iteration we factor the pageranks down by the status count. We get a new set of top users.

# Part 2: Learning to Rank (40 points)

For this part, we're going to play with some Microsoft LETOR data that has query-document relevance judgments. Let's see how learning to rank works in practice. 

First, you will need to download the MQ2008.zip file from the Resources tab on Piazza. This is data from the [Microsoft Research IR Group](https://www.microsoft.com/en-us/research/project/letor-learning-rank-information-retrieval/).

The data includes 15,211 rows. Each row is a query-document pair. The first column is a relevance label of this pair (0,1 or 2--> the higher value the more related), the second column is query id, the following columns are features, and the end of the row is comment about the pair, including id of the document. A query-document pair is represented by a 46-dimensional feature vector. Features are a numeric value describing a document and query such as TFIDF, BM25, Page Rank, .... You can find compelete description of features from [here](https://arxiv.org/ftp/arxiv/papers/1306/1306.2597.pdf).

The good news for you is the dataset is ready for analysis: It has already been split into 5 folds (see the five folders called Fold1, ..., Fold5).


## Part 2.1: Build Point-wise Learning to Rank  (20 points)
First, you should build a point-wise Learning to Rank framework. 
1. You could train a binary classification model like SVM or logistic regression on the train file. In this case, 0 is treated as negative (irrelevant) sample and 1, 2 are treated as positive (relevant) sample.
2. You apply the already trained model to predict the scores for documents on test file.
3. Order the documents based on the scores.

In [11]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import pandas as pd
import math 

In [12]:
# Function to transform the data
# 0 -> Query id
# 1 -> Document id
# 2-47 -> Features
# 48 -> ranking
# 49 -> label
def transformLETOR(input_df):
    input_df[96] = input_df[0]
    input_df[0] = input_df[2]
    input_df[2] = input_df[97]
    drop_cols = list(range(1,96,2))
    drop_cols.extend(range(97,104))
    input_df_trans = input_df.drop(drop_cols,1)
    input_df_trans.columns = list(range(0,49))
    input_df_trans[49] = input_df_trans[48] > 0
    input_df_trans.infer_objects()
    input_df_trans[49] = input_df_trans[49].apply(int)
    
    return input_df_trans

    

# Create a SVC classifier using an RBF kernel
def learn_to_rank(Folder):
    
    train_df_raw = pd.read_csv(Folder + "train.txt"," |:",header=None,engine='python')
    train_df = transformLETOR(train_df_raw)
    
    val_df_raw = pd.read_csv(Folder + "vali.txt"," |:",header=None, engine='python')
    val_df = transformLETOR(val_df_raw)
    
    test_df_raw = pd.read_csv(Folder + "test.txt"," |:",header=None, engine='python')
    test_df = transformLETOR(test_df_raw)
    
    X_train = train_df.iloc[:,2:48]
    Y_train = train_df.iloc[:,49]
    
    X_val = val_df.iloc[:,2:48]
    
    X_test = test_df.iloc[:,2:48]
    
    results = []
    
    C = [1e-5,1e-4,1e-3,1e-2,1e-1,1e0,1e1]
    
    for hyp in C:
        logreg_class = LogisticRegression(C=hyp,max_iter = 500, class_weight = 'balanced',solver='lbfgs')
        
        # Train the classifier
        logreg_class.fit(X_train, Y_train)
        predictions = logreg_class.predict_proba(X_val)
        val_df[51] = predictions[:,1]
        
        val_ndgc = NDCG(val_df)
        results.append(val_ndgc)
    
    best_hyp = results.index(max(results))
    logreg_class = LogisticRegression(C=C[best_hyp],max_iter = 500, class_weight = 'balanced',solver='lbfgs')
    logreg_class.fit(X_train, Y_train)
    predictions = logreg_class.predict_proba(X_test)
    test_df[51] = predictions[:,1]
    test_ndgc = NDCG(test_df)
    
    return test_ndgc

I am using Logistic Regression for learning a classifier. The approach used to tune the hyperparameter is to fit on training data and predict validation data. The best NDGC score for validation data is the winner and we compute the test scores with that hyperparameter.

## Part 2.2: NDCG (20 points)

Based on your prediction file (results could be ranked by scores in the prediction file) and ground-truth (i.e., 0,1,2) in the test file, calculate NDCG for each query. Report average NDCG for all queries in the five-fold cross validation.

For NDCG, please bulid your own function rather then using any package.

In [13]:
# your code here
# pred_df needs to have query_id,document_id,score
def NDCG(pred_df):
    final = pred_df.sort_values([0,51],ascending=[False, False])    

    query_id = final[0][0]
    user_rel = []
    
    ndgc = 0
    query_count = 0
    for j in range(final.shape[0]):
        if query_id != final[0][j]:
            # get ndcg
            dgc = 0
            for k in range(min(10,len(user_rel))):
                dgc += user_rel[k]/(math.log(k+2))
                
            ideal_rel = sorted(user_rel,reverse=True)
            idgc = 0
            
            for k in range(min(10,len(user_rel))):
                idgc += (ideal_rel[k])/(math.log(k+2))                
            
            if idgc:
                ndgc += (dgc/idgc)
            
            query_id = final[0][j]
            user_rel = [final[48][j]]
            query_count += 1
        else:
            if len(user_rel)<10:
                user_rel.append(final[48][j])
    
    return ndgc/query_count

In [14]:
for idx in range(1,6):
    directory = "MQ2008/MQ2008/Fold"+str(idx)+"/"
    dir_ndgc = learn_to_rank(directory)
    print("NDGC for fold"+str(idx)+": ",dir_ndgc)

NDGC for fold1:  0.36612722571674816
NDGC for fold2:  0.3939720988916711
NDGC for fold3:  0.36553542096652986
NDGC for fold4:  0.4383451079899545
NDGC for fold5:  0.421025156153727


## (BONUS) Pairwise Learning to Rank (5 points)

Rather than use the point-wise approach as in Part 2.1, instead try to implement a paiwise approach.

In [15]:
# your code here

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*